<a href="https://colab.research.google.com/github/gdoorash/Cassava-Disease-Classification/blob/master/Notebooks/ResNet_Pretrained_cassava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ResNet_Pretrained Architecture**

In [0]:
import numpy
import torch 
import torchvision 
import os
import matplotlib.pyplot as plt
import random 
import pdb
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, sampler
import imageio
import PIL
from IPython import display
from torchvision import datasets
import torch.nn as nn
 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip "/content/gdrive/My Drive/Colab Notebooks/Data/Cassava_t_v_t.zip";

In [0]:
%matplotlib inline

def show(img):
    """Show PyTorch tensor img as an image in matplotlib."""
    npimg = img.cpu().detach().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation='nearest')
    plt.grid(False)
    plt.gca().axis('off')

def display_thumb(img):
  display.display(transforms.Resize(128)(img))

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## **Prepare the dataset** (balancing, spiliting, data augmentation, ..etc)

In [0]:
# Load Cassava dataset
traindata_folder_path = '/content/ammi-2020-convnets/train/train'
valdata_folder_path = '/content/ammi-2020-convnets/train/validation'
testdata_folder_path = '/content/ammi-2020-convnets/test/test'


# load the extra data for self-supervie (Autoencodrer)..
extra_folder_path = '/content/ammi-2020-convnets/extraimages'


In [0]:
# compute the mean and standard deviation of the all images (train, val, test)

In [0]:
# torch_example = torch.randn((100, 3, 50, 50))
# torch_example_mean = torch_example.mean(axis=(0,2,3))
# torch_example_mean
# # torch_example_mean2 = torch_example.mean(axis=(2,3))
# print('per batch = {0}'.format(torch_example_mean2.shape))
# torch_example_mean2 = torch_example_mean2.mean(axis=(0))
# print('per batch_and h and w = {0}'.format(torch_example_mean2.shape))


tensor([ 0.0001, -0.0008, -0.0002])

In [0]:
# tf = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.ToTensor(),
# ])
# cassava_train = datasets.ImageFolder(traindata_folder_path, transform=tf)
# trainloader = torch.utils.data.DataLoader(cassava_train, batch_size=700, shuffle=True, num_workers=8)

In [0]:
# # compute the mean and standard deviation of the images ...
# run_mean = torch.zeros(3)
# for i, (batch,_) in enumerate(trainloader):
#   curr_mean = batch.mean(axis=(0,2,3))
#   run_mean = ((run_mean*i) + curr_mean)/(i+1) 
  


In [0]:
# run_mean 

tensor([0.4479, 0.4963, 0.3217])

In [0]:

# for imgs,_ in trainloader:
#     print(imgs.std((0,2,3)))
#     break

tensor([0.2074, 0.2087, 0.1896])


In [0]:
######################### Transformations of images #################################################


inference_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.4479, 0.4963, 0.3217], [0.2074, 0.2087, 0.1896])
])

# For visualization purposes we'll create a separate transform that operates in image space.
inference_transform_show = transforms.Compose([
    transforms.Resize(500),
    transforms.CenterCrop(256),
])


########################################   Transform and put the data into Dataset object #########################################################################

transformed_cassava_trian = datasets.ImageFolder(traindata_folder_path, transform=inference_transform)
transformed_cassava_validation = datasets.ImageFolder( valdata_folder_path, transform=inference_transform) 
transformed_cassava_test = datasets.ImageFolder(testdata_folder_path,  transform=inference_transform)

##########################################  Load the data into DataLoader #########################################################
transformed_cassava_trainloader = torch.utils.data.DataLoader(transformed_cassava_trian, batch_size=32, shuffle=True, num_workers=8)
transformed_cassava_valloader = torch.utils.data.DataLoader(transformed_cassava_validation, batch_size=32, shuffle=False, num_workers=8)
transformed_cassava_testloader = torch.utils.data.DataLoader(transformed_cassava_test, batch_size=32, shuffle=False, num_workers=8)




########## Extra data set loading ##########################################################################

transformed_cassava_extra = datasets.ImageFolder(extra_folder_path, transform=inference_transform)
transformed_cassava_extraloader = torch.utils.data.DataLoader(transformed_cassava_extra , batch_size=32, shuffle=True, num_workers=8)



## **Basic function for training & testing**

In [0]:
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [0]:
train_losses = []
valid_losses = []

def train (model, criterion, train_dataloader, valid_dataloader, optimizer, num_epochs):
    
    
    
    # move the model to GPU
    model.to(device)
    
    




    print("# Start training...")
    for epoch in range(num_epochs):
      
      train_loss = 0
      train_n_iter = 0
      tcorrect = 0

      # put the model in train mode
      model.train()
     
      for batch_index, (batch, target) in enumerate(train_dataloader):
          batch = batch.to(device)
          target = target.to(device)
          
          optimizer.zero_grad()
          
          # Forward step 
          output = model(batch)
          loss = criterion(output.to(device), target)
          
          # Backward step 
          loss.backward()
          #gradient step
          optimizer.step()
          

          # compute the accuracy
          prediction = output.argmax(dim=1, keepdim=True)
          tcorrect += prediction.cpu().eq(target.cpu().view_as(prediction)).sum().item()

          # avarge of loss
          train_loss += loss.item()
          train_n_iter += 1

      valid_loss = 0
      valid_n_iter = 0
      vcorrect = 0

      model.eval()

      #iterate over valid data..
      for images, labels in transformed_cassava_valloader:
        images = images.to(device)
        labels = labels.to(device)

        #forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # compute accuracy
        prediction = outputs.argmax(dim=1, keepdim=True)
        vcorrect += prediction.cpu().eq(labels.cpu().view_as(prediction)).sum().item()
        
        # loss and accuracy
        valid_loss += loss.item()
        valid_n_iter += 1

      
      train_losses.append(train_loss/train_n_iter)
      valid_losses.append(valid_loss/valid_n_iter)
   
      train_percent = 100. * tcorrect / len(train_dataloader.dataset)
      valid_percent = 100. * vcorrect / len(valid_dataloader.dataset) 

      # Print out progress the end of epoch.
      print('\nEpoch:{}/{} \tTrain Loss: {:.4f} \tValid Loss: {:.4f} -------- Train Accuracy: {} \tValid Accuracy: {}\tDF: {}'.format(
         epoch + 1, num_epochs,
        train_loss/train_n_iter, valid_loss / valid_n_iter,
        train_percent, valid_percent, train_percent-valid_percent))

def test (model, criterion, dataloader):
    
    model.eval()
    correct = 0
    # avg_loss = 0
    with torch.no_grad():
        
        for data, target in dataloader:
             
            data = data.to(device)
            target = target.to(device)
            
            output = model(data).to(device)
            loss = criterion(output, target) 
            
            prediction = output.argmax(dim=1, keepdim=True) 
            correct += prediction.cpu().eq(target.view_as(prediction)).sum().item()
            
            # avg_loss += loss.item() 
    percent = 100. * correct / len(dataloader.dataset)
    print(f'Accuracy: {correct}/{len(dataloader.dataset)} ({percent:.0f}%)')
   
    return percent  
    
    
    
        
        
            

## **Model Architecture (ResNet)** 

In [73]:
# define the per-trained model...
full_resnet_model = torchvision.models.resnet18(pretrained=True)
full_resnet_model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
class ModelResNet(nn.Module):

  def __init__(self):
    super(ModelResNet, self).__init__()
    self.res_mod = nn.Sequential(*list(full_resnet_model.children())[:-5])
    self.avgpool = nn.AdaptiveAvgPool2d(1)
    self.fc =  nn.Linear(64, 5)

  def forward(self, x):
      x = self.res_mod(x)
      x = self.avgpool(x)
      x = self.fc(x.view(x.size(0), -1))
      return x

In [81]:
# Model Architectuer..
model = ModelResNet()
model

ModelResNet(
  (res_mod): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

## **Run the Model**  (checkpoint save and load, optimizer,...)

In [0]:
# Create checkpoint ...
path = "/content/gdrive/My Drive/Colab Notebooks/model_data_checkpoints/ResNet/resnetcp_1.pt"
torch.save(model.state_dict(), path)

In [0]:
# load the checkpoint...
# model = model.load_state_dict(torch.load("/content/gdrive/My Drive/Colab Notebooks/model_data_checkpoints/ResNet/resnetcp_1.pt"))
# model

In [82]:
if device == 'cuda':
    model = model.cuda()
num_epochs = 100   
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
train(model, nn.CrossEntropyLoss(), transformed_cassava_trainloader,  transformed_cassava_valloader, optimizer, num_epochs)

# Start training...

Epoch:1/100 	Train Loss: 1.1133 	Valid Loss: 1.7727 -------- Train Accuracy: 60.32639279684862 	Valid Accuracy: 36.30769230769231

Epoch:2/100 	Train Loss: 1.0497 	Valid Loss: 1.7286 -------- Train Accuracy: 62.16469705496154 	Valid Accuracy: 36.61538461538461

Epoch:3/100 	Train Loss: 1.0282 	Valid Loss: 1.7875 -------- Train Accuracy: 63.44025511161133 	Valid Accuracy: 35.69230769230769

Epoch:4/100 	Train Loss: 1.0081 	Valid Loss: 1.8721 -------- Train Accuracy: 63.98424310635903 	Valid Accuracy: 33.53846153846154

Epoch:5/100 	Train Loss: 0.9888 	Valid Loss: 1.7257 -------- Train Accuracy: 64.30313262052148 	Valid Accuracy: 40.30769230769231

Epoch:6/100 	Train Loss: 0.9840 	Valid Loss: 1.4978 -------- Train Accuracy: 65.01594447570812 	Valid Accuracy: 40.61538461538461

Epoch:7/100 	Train Loss: 0.9689 	Valid Loss: 2.3509 -------- Train Accuracy: 64.75332958169199 	Valid Accuracy: 36.30769230769231

Epoch:8/100 	Train Loss: 0.9501 	Valid Loss: 1.4499 -------- T

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Process Process-880:


Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-82-d4c3618c3f6e>", line 5, in <module>
    train(model, nn.CrossEntropyLoss(), transformed_cassava_trainloader,  transformed_cassava_valloader, optimizer, num_epochs)
  File "<ipython-input-58-b9daa66e84a3>", line 24, in train
    for batch_index, (batch, target) in enumerate(train_dataloader):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 345, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 841, in _next_data
    idx, data = self._get_data()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 808, in _get_data
    success, data = self._try_get_data()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.

KeyboardInterrupt: ignored

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 261, in _bootstrap
    util._exit_function()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 322, in _exit_function
    _run_finalizers()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 262, in _run_finalizers
    finalizer()
  File "/usr/lib/python3.6/multiprocessing/util.py", line 186, in __call__
    res = self._callback(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 191, in _finalize_join
    thread.join()
  File "/usr/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.6/threading.py", line 1072, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


In [84]:
# Save history for later
train_loss_history = train_losses
valid_loss_history = valid_losses

# Plot training and validation curve
x = range(1, num_epochs + 1)
plt.plot(x, train_loss_history, label='train')
plt.plot(x, valid_loss_history, label='valid')

plt.xlabel('# epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

NameError: ignored